# Day 23: Safe Cracking

https://adventofcode.com/2016/day/23

A modified version of the `assembunny` code from Day 12, inclusing a new `toggle` instruction that modify program during executions...

In [191]:
def assembunny(_prog,a=0,b=0,c=0,d=0,verbose=False): 
    prog = list(_prog) # make a copy of the program to facilitate reuse and testing
    reg = { 'a': a, 'b': b, 'c': c, 'd': d }
    i = 0
    while True:
        if i>=len(prog):
            break
        cmd = prog[i].split(" ")
        if cmd[0]=="cpy":
            v = 0
            try:
                v = int(cmd[1])
            except:
                v = reg[cmd[1]]
            # check if instruction is valid after toggling (e.g. cmd[2] should be a register)
            if cmd[2] in ["a","b","c","d"]:
                reg[cmd[2]] = v
            i+=1
        elif cmd[0]=="inc":
            reg[cmd[1]] += 1
            i+=1
        elif cmd[0]=="dec":
            reg[cmd[1]] -= 1
            i+=1
        elif cmd[0]=="jnz":
            v= 0 
            try:
                v = int(cmd[1])
            except:
                v = reg[cmd[1]]
            if v!= 0:
                try:
                    i += int(cmd[2])
                except:
                    i += reg[cmd[2]]
            else:
                i += 1 
        elif cmd[0]=="tgl":
            # get the instruction to be toggled
            v = 0
            try:
                v = int(cmd[1])
            except:
                v = reg[cmd[1]]
            if i+v>=len(prog): # outside the program, do nothing
                pass
            else:
                ctgl = prog[i+v].split(" ")
                if len(ctgl)==2: # one argument
                    if ctgl[0]=="inc":
                        ctgl[0] = "dec"
                    else:
                        ctgl[0] = "inc"
                else:
                    if ctgl[0]=="jnz":
                        ctgl[0] = "cpy"
                    else:
                        ctgl[0] = "jnz"
                prog[i+v] = " ".join(ctgl)
            i+=1
        if verbose:
            print("{}  {} \t {}".format(i-1,prog[i-1],reg))
    return reg

In [192]:
prog = [
"cpy 2 a",
"tgl a",
"tgl a",
"tgl a",
"cpy 1 a",
"dec a",
"dec a"
]

In [193]:
assembunny(prog,verbose=True) 

0  cpy 2 a 	 {'a': 2, 'b': 0, 'c': 0, 'd': 0}
1  tgl a 	 {'a': 2, 'b': 0, 'c': 0, 'd': 0}
2  tgl a 	 {'a': 2, 'b': 0, 'c': 0, 'd': 0}
3  inc a 	 {'a': 3, 'b': 0, 'c': 0, 'd': 0}
6  dec a 	 {'a': 3, 'b': 0, 'c': 0, 'd': 0}


{'a': 3, 'b': 0, 'c': 0, 'd': 0}

## Part 1

In [194]:
with open('data/input23.txt') as f:
    prog = [ l.strip("\n") for l in f.readlines() ] 

reg = assembunny(prog,a=7)
print(reg['a'])

10584


## Part 2

Execution time grows dramatically, and the puzzle test suggests addition should be replaced by multiplication.

After some digging, I understand this simple assembry code implement multiplication as series of addition +1. This is definitively not very efficient, especially given what the code is trying to do. More trial-and error seems in fact to indicate that the code compute the factorial of the value input to register `a`: obviously the execution time explodes!

Ideally, I should implement some optimization of the assembly code to replace the loop implementing the multiplication as series of addition with a real multiplication. Unfortunately, my understanding of assembly of compiler optimisation is almost null.

On the other hand, a few tests has lead me to understand what the program does: given the input `N` in register `a`, it computes:

`factorial(N) + 72*77`

where the values `72` and `77` are stored in these instructions in my version of the original code:

`cpy 72 c`

`jnz 77 d`

Note the these instructions get modified by the previous `tgl` instruction to obfuscate what the program is doing even more!

In [195]:
import math

reg = assembunny(prog,a=7)

print(reg['a'],'=',math.factorial(7)+72*77)

10584 = 10584


So this should be the solution to Part 2:

In [196]:
print(math.factorial(12)+72*77)

479007144


I would anyway try to brute-force the solution using the basic assembly implementation before submitting the answer... Given the timing results for the lower value of `n`, I estimate around one hour of execution time.

BTW, I noticed that the execution time also esplodes for n<=5, I'm wondering if the code works in the same way for those values. Maybe at some point I'll check...

In [213]:
from timeit import default_timer as timer
start = timer()

def checkResults(n,prog):
    reg = assembunny(prog,a=n,verbose=False)
    return reg['a']

for n in range(6,13):
    r = checkResults(n,prog)
    lap = timer()
    sec = int(lap-start)
    print("{0:3d} | {1:15d} | {2:15d} | {3:8d}'{4:2d}''".format(n,r,math.factorial(n)+72*77,sec//60,sec%60))
    start = lap

  6 |            6264 |            6264 |        0' 0''
  7 |           10584 |           10584 |        0' 0''
  8 |           45864 |           45864 |        0' 0''
  9 |          368424 |          368424 |        0' 2''
 10 |         3634344 |         3634344 |        0'27''
 11 |        39922344 |        39922344 |        5' 3''
 12 |       479007144 |       479007144 |       73'57''
